# Pirates! Part 3

This is the analytic part of the Pirates! case study.

In the previous parts (completed in SAP) you have:
1. purchased a special raw material from overseas and add a standard security surcharge).
2. built a 'haystack' of legitimate purchase transactions.
3. included fraudulent purchase transactions. 

Now, your **objective** is to complete the following tasks:

1. Select and prepare the relevant tables for your analysis (1)
2. Develop an overview of the purchase transactions (1)
3. Identify, describe, and explain irregularities in the data (1)
4. Show the fraudulent transactions (2)
    * What is the amount of damage?
    * Which vendor is the 'problematic' one?

**CAUTION**:
You already know what happenend. You are not allowed to take shortcuts. Your analysis should show the full analytic process (prepare, overview, irregularities, fraud).

You have requested a copy of the potentially relevant tables from the company's SAP system. You find the previous case studies [here](https://github.com/mschermann/forensic_accounting). The cheat sheet is [here](https://github.com/mschermann/forensic_accounting/blob/master/Cheat%20Sheet.md). 

**Can you establish sufficient evidence for fraud?**

## Setup

Some initialization to make life easier. **Make sure to run the following cell before proceeding.**

In [1]:
#Allow multiple outputs for each cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
#Show simple plots in the notebook
import matplotlib.pyplot as plt
%matplotlib inline

We use the following libraries:
* [Pandas](https://pandas.pydata.org) is the most important workhorse in data analytics.
* [Altair](https://altair-viz.github.io) is a visualization library.

In [2]:
import pandas as pd
import altair as alt
from altair import *
import numpy as np
#Format number without any decimals
pd.set_option('display.float_format', lambda x: '%.4f' % x)
#Show altair plots in the notebook
alt.renderers.enable('notebook')

## Data

You have received the following tables from the SAP system:
1. The table `EKKO` contains header information of purchase orders.
2. The table `EKPO` contains line item information of the purchase orders.
2. The table `KONV` contains the conditions associated with purchase orders.
2. The table `MKPF` contains the header information of the material documents.
3. The table `MSEG` contains the line item information of the material documents.
2. The table `BKPF` contains the header information of the financial documents.
3. The table `BSEG` contains the line item information of the financial documents.

### The EKKO table

For our purposes, we use the following columns from `EKKO`:
* `EBELN` - Contains the purchase order number.
* `ERNAM` - Contains the purchase group employee who authorized the purchase order.
* `LIFNR` - Contains the vendor unique identifier that received the purchase order.
* `KNUMV` - Contains the link to the set of conditions associated with the purchase order.

In [3]:
ekko_table = pd.read_csv('https://raw.githubusercontent.com/mschermann/forensic_accounting/master/EKKO_Pirates.csv')

In [4]:
ekko_table.head()

,MANDT,EBELN,BUKRS,BSTYP,BSART,BSAKZ,LOEKZ,STATU,AEDAT,ERNAM,...,MEMORY,RLWRT,REVNO,RETTP,RETPC,DPTYP,DPPCT,DPAMT,DPDAT,VSART
0,670,4500000000,US00,F,NB,NaN,nan,9,05/03/2017,COCCHI,...,NaN,0.0000,nan,nan,0.0000,nan,0.0000,0.0000,00/00/0000,nan
1,670,4500000001,US00,F,NB,NaN,nan,9,04/23/2018,GBIFAC-01,...,NaN,0.0000,nan,nan,0.0000,nan,0.0000,0.0000,00/00/0000,nan
2,670,4500000002,US00,F,NB,NaN,nan,9,04/23/2018,GBIFAC-01,...,NaN,0.0000,nan,nan,0.0000,nan,0.0000,0.0000,00/00/0000,nan
3,670,4500000003,US00,F,NB,NaN,nan,9,04/26/2018,GBI-027,...,NaN,0.0000,nan,nan,0.0000,nan,0.0000,0.0000,00/00/0000,nan
4,670,4500000004,US00,F,NB,NaN,nan,9,04/27/2018,GBI-011,...,NaN,0.0000,nan,nan,0.0000,nan,0.0000,0.0000,00/00/0000,nan


### The EKPO table

For our purposes, we use the following columns from `EKPO`:
* `EBELN` - Contains the purchase order number.
* `EBELP` - Contains the line item identifier.
* `TXZ01` - Contains a textual description of the material.
* `MATNR` - Contains the material unique identifier.
* `MENGE` - Contains the amount of material ordered.
* `NETPR` - Contains the effective net price of material ordered.
* `NETWR` - Contains the effective net value of material ordered (i.e., `MENGE * NETPR`).

In [5]:
ekpo_table = pd.read_csv('https://raw.githubusercontent.com/mschermann/forensic_accounting/master/EKPO_Pirates.csv')

In [6]:
ekpo_table.head()

,MANDT,EBELN,EBELP,LOEKZ,STATU,AEDAT,TXZ01,MATNR,EMATN,BUKRS,...,ARSPS,INSNC,SSQSS,ZGTYP,EAN11,BSTAE,REVLV,GEBER,FISTL,FIPOS
0,670,4500000000,10,NaN,nan,05/03/2017,Off Road Helmet,OHMT1999,OHMT1999,US00,...,0,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,670,4500000000,20,NaN,nan,05/03/2017,Road Helmet,RHMT1999,RHMT1999,US00,...,0,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,670,4500000001,10,NaN,nan,04/23/2018,Infinity Stone,INFINITY-STONE-001,INFINITY-STONE-001,US00,...,0,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,670,4500000002,10,NaN,nan,04/23/2018,Tesseract,TESSERACT,TESSERACT,US00,...,0,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,670,4500000003,10,NaN,nan,04/26/2018,Kryptonite,KRYPTONITE-027,KRYPTONITE-027,US00,...,0,nan,nan,nan,nan,nan,nan,nan,nan,nan


### The KONV table

For our purposes, we use the following columns from `KONV`:
* `KNUMV` - Contains the unique identifier for the condition set.
* `KPOSN` - Contains the line item identifier.
* `KSCHL` - Contains the type of a condition.
* `KAWRT` - Contains the baseline of a conditon.
* `KBETR` - Contains the effective price.

In [7]:
konv_table = pd.read_csv('https://raw.githubusercontent.com/mschermann/forensic_accounting/master/KONV_Pirates.csv')

In [8]:
konv_table.head()

,MANDT,KNUMV,KPOSN,STUNR,ZAEHK,KAPPL,KSCHL,KDATU,KRECH,KAWRT,...,KWAEH,KFKIV,KVARC,KMPRS,PRSQU,STUFE,WEGXX,KTREL,MDFLG,TXJLV
0,670,1,10,11,1,V,PR00,05/02/2017,C,50.00,...,NaN,nan,nan,NaN,nan,0,0,nan,nan,nan
1,670,1,10,930,1,V,SKTO,05/02/2017,A,"15,000.00",...,NaN,nan,nan,NaN,nan,0,0,nan,nan,nan
2,670,1,10,940,1,V,VPRS,05/02/2017,C,50.00,...,USD,nan,nan,NaN,nan,0,0,nan,nan,nan
3,670,2,10,11,1,V,PR00,05/02/2017,C,20.00,...,NaN,nan,nan,NaN,nan,0,0,nan,nan,nan
4,670,2,10,930,1,V,SKTO,05/02/2017,A,"6,000.00",...,NaN,nan,nan,NaN,nan,0,0,nan,nan,nan


### The MKPF table

For our purposes, we use the following columns from `MKPF`:
* `MBLNR` - Contains the material document number.
* `USNAM` - Contains the inventory employee who posted the material document.

In [9]:
mkpf_table = pd.read_csv('https://raw.githubusercontent.com/mschermann/forensic_accounting/master/MKPF_Pirates.csv')

In [10]:
mkpf_table.head()

,MANDT,MBLNR,MJAHR,VGART,BLART,BLAUM,BLDAT,BUDAT,CPUDT,CPUTM,...,FRATH,FRBNR,WEVER,XABLN,AWSYS,BLA2D,TCODE2,BFWMS,EXNUM,KNUMV
0,670,4900002060,2018,WA,WA,PR,04/23/2018,04/23/2018,04/23/2018,16:37:23,...,0.0000,nan,2.0000,nan,TORCLNT670,nan,MB1C,nan,nan,nan
1,670,4900002061,2018,WA,WA,PR,04/23/2018,04/23/2018,04/23/2018,16:42:23,...,0.0000,nan,2.0000,nan,TORCLNT670,nan,MB1A,nan,nan,nan
2,670,4900002062,2018,WL,WL,PR,04/23/2018,04/23/2018,04/23/2018,16:47:59,...,0.0000,nan,nan,nan,TORCLNT670,nan,VL02N,2.0000,nan,nan
3,670,4900002063,2018,WA,WA,PR,04/23/2018,04/23/2018,04/23/2018,17:25:21,...,0.0000,nan,2.0000,nan,TORCLNT670,nan,MB1C,nan,nan,nan
4,670,4900002064,2018,WA,WA,PR,04/23/2018,04/23/2018,04/23/2018,17:35:44,...,0.0000,nan,2.0000,nan,TORCLNT670,nan,MB1C,nan,nan,nan


### The MSEG table

For our purposes, we use the following columns from `MSEG`:
* `MBLNR` - Contains the material document number.
* `BWART` - Contains the movement type of the line item. This [link](https://wiki.scn.sap.com/wiki/display/ERPLO/Movement+types) contains information about the movement types.
* `MATNR` - Contains the material id of the material moved.
* `WERKS` - Contains the plant id.
* `LGORT` - Contains the storage location.
* `DMBTR` - Contains the value of the material movement.
* `MENGE` - Contains the volume of the movement in units of the material. 

In [11]:
mseg_table = pd.read_csv('https://raw.githubusercontent.com/mschermann/forensic_accounting/master/MSEG_Pirates.csv')

In [12]:
mseg_table.head()

,MANDT,MBLNR,MJAHR,ZEILE,BWART,XAUTO,MATNR,WERKS,LGORT,CHARG,...,BESTQ,BWLVS,TBNUM,TBPOS,XBLVS,VSCHN,NSCHN,DYPLA,UBNUM,TBPRI
0,670,4900002060,2018,1,561,NaN,TRWA1001,DL00,SF00,nan,...,nan,0,0,0,NaN,NaN,NaN,NaN,0,nan
1,670,4900002060,2018,2,561,NaN,TRFR2001,DL00,RM00,nan,...,nan,0,0,0,NaN,NaN,NaN,NaN,0,nan
2,670,4900002060,2018,3,561,NaN,DGAM1001,DL00,RM00,nan,...,nan,0,0,0,NaN,NaN,NaN,NaN,0,nan
3,670,4900002060,2018,4,561,NaN,TRSK1001,DL00,RM00,nan,...,nan,0,0,0,NaN,NaN,NaN,NaN,0,nan
4,670,4900002060,2018,5,561,NaN,TRHB1001,DL00,RM00,nan,...,nan,0,0,0,NaN,NaN,NaN,NaN,0,nan


### The BKPF table

For our purposes, we use the following columns from `BKPF`:
* `BELNR` - That is the financial document number
* `BLART` - That is the document type
* `USNAM` - That is the user who posted the financial document

In [13]:
bkpf_table = pd.read_csv('https://raw.githubusercontent.com/mschermann/forensic_accounting/master/BKPF_Pirates.csv')

In [14]:
bkpf_table.head()

,MANDT,BUKRS,BELNR,GJAHR,BLART,BLDAT,BUDAT,MONAT,CPUDT,CPUTM,...,CTXKRS,LOTKZ,XWVOF,STGRD,PPNAM,PPDAT,PPTME,BRNCH,NUMPG,ADISC
0,670,US00,90000003,2018,RV,04/23/2018,04/23/2018,4,04/23/2018,16:49:20,...,0.0000,nan,nan,nan,nan,00/00/0000,00:00:00,nan,0,nan
1,670,US00,90000004,2018,RV,05/02/2018,05/02/2018,5,05/02/2018,16:29:22,...,0.0000,nan,nan,nan,nan,00/00/0000,00:00:00,nan,0,nan
2,670,US00,90000005,2018,RV,05/11/2018,05/11/2018,5,05/11/2018,17:05:19,...,0.0000,nan,nan,nan,nan,00/00/0000,00:00:00,nan,0,nan
3,670,US00,90000006,2018,RV,05/12/2018,05/12/2018,5,05/12/2018,12:41:57,...,0.0000,nan,nan,nan,nan,00/00/0000,00:00:00,nan,0,nan
4,670,US00,90000007,2018,RV,05/13/2018,05/13/2018,5,05/13/2018,00:16:26,...,0.0000,nan,nan,nan,nan,00/00/0000,00:00:00,nan,0,nan


### The BSEG table

We use the following columns from `BSEG`:
* `BELNR` - That is the financial document number
* `BSCHL` - That is the posting key
* `WRBTR` - That is the amount posted
* `LIFNR` - That is vendor account

In [15]:
bseg_table = pd.read_csv('https://raw.githubusercontent.com/mschermann/forensic_accounting/master/BSEG_Pirates.csv')

In [16]:
bseg_table.head()

,MANDT,BUKRS,BELNR,GJAHR,BUZEI,BUZID,AUGDT,AUGCP,AUGBL,BSCHL,...,LIFNR,FILKD,XBILK,GVTYP,HZUON,ZFBDT,ZTERM,ZBD1T,ZBD2T,ZBD3T
0,670,US00,90000003,2018,1,NaN,04/23/2018,04/23/2018,1400000003.0000,1,...,nan,nan,X,NaN,nan,04/23/2018,1.0000,0,0,0
1,670,US00,90000003,2018,2,NaN,00/00/0000,00/00/0000,nan,50,...,nan,nan,NaN,RE,nan,00/00/0000,nan,0,0,0
2,670,US00,90000004,2018,1,NaN,05/02/2018,05/02/2018,1400000004.0000,1,...,nan,nan,X,NaN,nan,05/02/2018,1.0000,0,0,0
3,670,US00,90000004,2018,2,NaN,00/00/0000,00/00/0000,nan,50,...,nan,nan,NaN,RE,nan,00/00/0000,nan,0,0,0
4,670,US00,90000005,2018,1,NaN,05/11/2018,05/11/2018,1400000005.0000,1,...,nan,nan,X,NaN,nan,05/11/2018,1.0000,0,0,0


## Your investigation

The context of your investigation is as follows:
* Rumor has it that there are irregularites in your purchase processes.
* Your preliminary investigation suggests that none of your employees show suspicious behavior.
* However, from colleagues in other companies, you have heard of problems with specific vendors. 

Before you begin your investigation, it is vital to understand which tables will be relevant for your analysis. Use the following `markdown` cell to describe, which tables are important.

## Select and prepare the relevant tables for your analysis

## Develop an overview of the purchase transactions (1)

## Identify, describe, and explain irregularities in the data (1)

## Show the fraudulent transactions (2)

### What is the amount of damage?

### Which vendor is the 'problematic' one?

## Reflect: Can you explain the fraud?